# ASSIGNMENT 3                                            McGill:COMP766-2  Winter 2020 
Student ID

The objective of this assignment is to implement loopy belief propagation, and mean-field method for the Ising model, which we saw last time. Below is a review from the last assignment.

#### Ising model
The Ising model is fully expressed using a symmetric adjacency matrix $A$ (```adj```):
\begin{align}
  p(x) = \frac{1}{Z} \exp(\sum_{i} A_{i,i}x_i + \sum_{i,j < i} x_i A_{i,j}x_j) \quad x_i \in \{-1,+1\}
\end{align}
In words, the diagonal of $A$ represents the local field and interactions are captured by the remaining non-zero elements of $A$ (It is customary to have a negative sign in front of the summation, but for simplicity we dropped it.)
The graph structure can also be inferred from the sparsity patter of $A$.
Here, we are interested in deterministic sum-product inference.


## Problem 1.  Loopy Belief Propagation

### 1.a. (10 pts) 
Complete the implementation of loopy-BP below. (The base ```Inference``` class as well as ```CliqueTreeInference``` are included in the ```inference.py``` helper file.) 
Recall that the BP message $m_{i \to j}$ for a pairwise MRF is the product of all incoming messages, and the relevant factors, marginalized over $x_i$. For the Ising model this is:
\begin{align} 
m_{i \to j}(x_j) \propto (1-\alpha) m^{old}_{i \to j}(x_j) \, + \, \alpha \sum_{x_i \in \{-1,+1\}} \left ( \exp(x_i A_{i,i} + x_i A_{i,j} x_j) \prod_{k \in Nb(i)-j} m_{k \to i}(x_i) \right )
\end{align}
where $\alpha$ called the damping factor helps with the convergence. After convergence the marginals are:
\begin{align}
p(x_i) \propto \exp(x_i A_{i,i}) \prod_{j \in Nb(i)} m_{j \to i}(x_i)
\end{align}

In the implementation below:
Try to use only a single loop, which runs for a maximum of ```self.max_iters```. If you find this too challenging you can also loop over individual messages.
The iterations should terminate as soon as the maximum change in the absolute value of the marginals in two iterations goes below ```self.eps```. To help with convergence you can damp (smoothed) your message update using the damping parameter ```self.damping``` 

#### using log-ratio of messages
When dealing with a binary domain, it is more efficient to only track a single probability value (rather two values for the message $m_{i \to j}(-1), m_{i \to j}(+1)$ in the case of Ising model. Because this is redundant.) In fact, for numerical stability, it is convenient to track only the log-ratio $\mu_{i \to j} = \log \left( \frac{m_{i \to j}(+1)}{m_{i \to j}(-1)} \right )$. One could then try and rewrite the message update using $\mu$ rather original $m$. This enables us to only track $\mu$, and after convergence produce marginals using these log-ratios. we can also write the marginals for this setting. For this you need to substitute $m(1) = \frac{1}{1 + e^{-\mu}} = \sigma(\mu)$ and $m(-1) = 1 - \sigma(\mu)$ in the update equation above and simplify (if you decide __not__ to use log-ratios you may need to update other parts of the code below, which was written based on this assumption)

### 1.a. solution

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import networkx as nx
np.random.seed(12345)
import timeit
#removing some warnings!
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)
from utils import tensor_mult, draw_graph, logistic
from inference import Inference, CliqueTreeInference
from IPython.core.debugger import set_trace

class LBPInference(Inference):
    """
    implements sum-product loopy belief propagation inference 
    for  p(x) \propto exp(+\sum_{i,j<i} x_i x_j adj[i,j] + \sum_{i} x_i adj[i,i]) for  x_i \in {-1,+1}
    """
    def __init__(self,
                 adj,
                 verbosity=0,
                 max_iters=1000,  # max number of iterations
                 damping=.1,  # damping=0 means no damping
                 eps=1e-16,  # threshold for convergence
    ):
        Inference.__init__(self, adj, verbosity=verbosity)
        self.damping = damping
        self.max_iters = max_iters
        # (i,j)th element is the log-ratio of message i->j
        self.msg_new = None
        self.msg_old = None
        self.marginals = None
        self.converged = False
        self.mask = None  # zero-one mask for the adjacency (zero diagonals)
        self.eps = eps
        self._initialize()
        self.adj = adj

    def update_adjacency(self, new_adj):
        self.adj = new_adj
        self._initialize()

    def _initialize(self):
        n = self.adj.shape[0]
        self.msg_new = np.zeros((n , n))
        self.msg_old = np.zeros((n , n))
        self.marginals = np.zeros(n)
        mask = (self.adj != 0)
        np.fill_diagonal(mask, 0)
        self.mask = mask
        self.converged = False

    def _apply_damping(self, msg_new, msg_old):
        #convert to probability
        msg_new_p = logistic(msg_new)
        msg_old_p = logistic(msg_old)
        msg_p = msg_old_p * (self.damping) + msg_new_p * (1. - self.damping)
        # convert back to log-ratio
        msg = np.log(msg_p) - np.log(1. - msg_p)
        return msg
    
    def _inference(self):
        if self._verbosity > 0:
            print("running BP", flush = True)
        
        # >>>> YOUR CODE HERE >>>>>>>

        # >>>> YOUR CODE HERE >>>>>>>
        
        self.marginals = new_marginals
        if self._verbosity > 0:
            print("finished after {} iterations!".format(t), flush = True)
                
    def get_marginal(self, target):
        if not self.converged:
            self._inference()
            self.converged = True
        mrg = logistic(self.marginals)[target]
        return mrg


If the implementation above is correct, it should return exact marginals for the tree structure below. You can use this to debug your code (We are comparing the ```LBPInference``` marginals against the exact values returned by ```CliqueTreeInference```.) 

In [ ]:
#build a tree 
tree = nx.balanced_tree(3, 4)
n = nx.number_of_nodes(tree)
mask = np.array(nx.adjacency_matrix(tree).todense())
# create a random interaction over the edges
np.fill_diagonal(mask, 1.)
adj = mask * np.random.randn(n , n)
# make it symmetric
zero_inds = np.tril_indices_from(adj,-1)
adj[zero_inds] = 0        
adj += adj.T.copy()
# do inference
true_margs = CliqueTreeInference(adj).get_marginal(list(range(adj.shape[0])))            
bp_margs = LBPInference(adj).get_marginal(list(range(adj.shape[0])))
assert np.mean(np.abs(true_margs - bp_margs)) < 1e-10, "LBP marginals are not exact"

### 1.c. (5 pts) 
apply it to $n×n$ 2D-grids (```grid_2d_graph```), of increasing size $n \in \{2, 4, 6, 8, 15\}$,
and compare its run-time with clique-tree. The x-axis in the plot is the number of nodes $n$ and y-axis is the run-time (use logarithmic scale). Repeat your experiment at least 10 times to produce an error-bars for your plot.

### 1.c. solution

In [ ]:
# >>>> YOUR CODE HERE >>>>>>>

# >>>> YOUR CODE HERE >>>>>>>

## Problem 2. Mean Field (15 pts)

### 2.a. (8 pts)
Complete the implementation of naive mean-field below. 
You can refer to slides for derivation of the updates for the Ising model.
In the implementation below: 
Try to use only a single loop, which repeats for a a maximum of ```self.max_iters```. 
The loop can terminate as soon as the maximum of the absolute value of change in the marginals 
between two iterations goes below ```self.eps```.
For better convergence try damping (smoothing) the updates, using the damping parameter ```self.damping```.

### 2.a. solution

In [ ]:
class MFInference(Inference):
    """
    implements parallel update of naive mean-field for the Ising model
    mean-field is guaranteed to converge to a local optima, when updating 
    one coordinate at a time. Here, since for efficiency/simplicity we update all coordinates simultaneously,
    we may need to use damping to help with convergence
    for  p(x) \propto exp(+\sum_{i,j<i} x_i x_j adj[i,j] + \sum_{i} x_i adj[i,i]) for  x_i \in {-1,+1}
    """
    
    def __init__(self,
                 adj,  # adjacency matrix for the Ising model
                 verbosity=0,
                 max_iters=1000,  # max number of iterations
                 eps=1e-16,  # threshold for convergence
                 damping=.5, # because we are using parallel updates, 0 means no damping
    ):
        Inference.__init__(self, adj, verbosity=verbosity)
        self.damping = damping
        self.max_iters = max_iters
        # (i,j)th element is the log-ratio of message i->j
        self.means = None
        self.converged = False
        self.adj_masked = None  # adjacency matrix with zero diagonals
        self.eps = eps
        self._initialize()

    def update_adjacency(self, new_adj):
        self.adj = new_adj
        self._initialize()

    def _initialize(self):
        n = self.adj.shape[0]
        self.means = np.zeros(n)
        adj_masked = self.adj.copy()
        np.fill_diagonal(adj_masked, 0)
        self.adj_masked = adj_masked
        self.converged = False

    def _inference(self):
        if self._verbosity > 0:
            print("running MF", flush = True)
        # >>>> YOUR CODE HERE >>>>>>>

        # >>>> YOUR CODE HERE >>>>>>>
        if self._verbosity > 0:
            print("finished after {} iterations!".format(t), flush = True)
                
    def get_marginal(self, target):
        if not self.converged:
            self._inference()
            self.converged = True
        # converting the mean ([-1,+1] range) to the p(x_i = 1) [0,1] range
        mrg = (self.means[target] + 1.)/2.
        return mrg

### 2.b (7 pts)
Now, we can can compare the accuracy of belief propagation and mean-field in approximate inference.
For this experiment, we use a fixed $15\times15$ 2D grid (without periodic boundaries).
To control the difficulty of the inference problem, we adjust the temperature parameter $T$.
This corresponds to multiplication of the adjacency matrix $A$ with a constant. Large temperatures,
correspond to lower values of interaction $A_{ij}$, which reduces interdependency of variables:
\begin{align}
  p^{\frac{1}{T}}(x) = \frac{1}{Z(T)} \exp(\frac{1}{T}\sum_{i} A_{i,i}x_i + \frac{1}{T} \sum_{i,j < i} x_iA_{i,j}x_j) \quad x_i \in \{-1,+1\}
\end{align}
We sample the non-zero elements of $A$, from a standard normal distribution, and adjust
the difficulty of approximate inference by changing $T \in \{2^{-4},2^{-3},\ldots, 2^{9}\}$.
Use the mean-absolute-error of all the univariate marginals as a measure of accuracy of approximate inference, and plot temperature-vs-accuracy for both naive mean-field and loopy belief propagation.
Note that to measure the _error_, first you have to perform _exact inference_ on the same graphical model using ```CliqueTreeInference```, to obtain the exact marginals.
Repeat the experiment 10 times and produce error-bars for your difficulty-vs-accuracy plot (use logarithmic scale for both x and y).
Summarize what you see in the plot.

### 2.b solution
